## Connect to Database

In [1]:
import psycopg2
import datetime
import boto3
import os
import logging

In [2]:
# load_dotenv()

AWS_POSTGRES_PASSWORD = "Alex6581!"

In [3]:
def pg_connect():
    """Connect to postgres database instance.

    Returns:
        conn (object): psycopg2 connection object, to be used to interact with database
    """

    conn = psycopg2.connect(
        database="postgres",
        user="nexus_dev",
        password=AWS_POSTGRES_PASSWORD,  # Handle password with env file
        host="nexus-database.c0n44saktvjz.us-east-1.rds.amazonaws.com",
        port="5432",
    )
    # Connection details for AWS Relational Database Services Postgres Instance

    # session_logging.debug("Connected to Postgres Instance")

    return conn

In [4]:
conn = pg_connect()

## Fetch processed files

#### Remains the same

In [5]:
 def fetch_processed_files() -> list:
    """Fetches a list of file_ids that have already been processed.

    All file_ids returned by this function will not have to be processed further, as it has been handled at a previous data.

    Returns:
        file_ids (list): _description_
    """
    cursor = conn.cursor()  # TODO: Change to input variable
    cursor.execute("SELECT file_id from reonomy_silver_processed_files")
    fetched_ids = cursor.fetchall()
    file_ids = [id_[0] for id_ in fetched_ids]

    return file_ids

In [6]:
file_ids = fetch_processed_files()

## Fetch all files

In [7]:
aws_bucket_name = "nexus-equities-reonomy-bronze-bucket"
gcp_bucket_name = "nexus-reonomy-development-raw"

### AWS

In [8]:
def fetch_all_files_aws() -> list:
    """Fetches all file names from the S3 bronze bucket instance.

    All files fetched from this instance will be compared to processed files to decide which files to process in the future.

    Returns:
        all_files (list): List of all files from s3 bronze bucket zone.
    """
    s3 = boto3.resource("s3")
    my_bucket = s3.Bucket(bucket_name)  # TODO: Change to global variable

    all_files = []
    for object_summary in my_bucket.objects.filter(Prefix=f"scrapy/"):
        key = object_summary.key
        if key != "scrapy/":
            all_files.append(key)

    return all_files



### GCP

In [8]:
from google.cloud import storage

/Users/alexandergirardet/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [9]:
from google.cloud import storage

def fetch_all_files(delimiter=None):
    """Fetches all file names from the GCP raw bucket instance.

    All files fetched from this instance will be compared to processed files to decide which files to process in the future.

    Returns:
        all_files (list): List of all files from GCP raw bucket zone.
    """

    client = storage.Client.from_service_account_json(json_credentials_path='./develop_gcp_service_account_key.json')

    blobs = client.list_blobs(bucket_name, prefix=f"scrapy/", delimiter=delimiter)
    
    all_files = []

    for blob in blobs:
        name = blob.name
        if name != "scrapy/":
            all_files.append(name)
    return all_files

In [10]:
def file_name_to_id(file_list: list) -> list:
    """Takes the full file string and returns a list of file ids.
    These cleaned file ids will be compared to processed files.

    Args:
        file_list (list): all_files containing full strings as returned by boto, that need to be cleaned.

    Returns:
        cleaned_file_id_list (list): list of files only containing file_ids.
    """

    file_id_list = [file.split("/")[-1] for file in file_list]

    cleaned_file_id_list = [file.split(".")[0] for file in file_id_list]

    return cleaned_file_id_list

def get_remaining_files(all_files: list, processed_file_ids: list) -> list:
    """Returns all files that have yet to be processed.

    Uses file_name_to_id to process all_files to all_file_ids, containing only the id strings, to compare with process files.
    As boto only takes full file names we must zip the file ids and full file names together, check if the ids have already been processed,
    and store non processed ids in files_remaining as the full id string.

    Args:
        all_files (list): all files from s3 bronze zone as full strings
        processed_file_ids (list): all files from postgres instance that have already been processed as file ids

    Returns:
        files_remaining (list): A list of full s3 url files that have yet to be processed.
    """

    all_files_ids = file_name_to_id(all_files)

    files_remaining = []
    for full_file, file_id in zip(all_files, all_files_ids):
        if file_id not in processed_file_ids:
            files_remaining.append(full_file)

    return files_remaining

# Batch functions

def get_batch_data(files_remaining: list) -> list:
    """Batches files into groups of 5.

    As each file contains 50 property objects, we do not want to overload the functions at a given time and
    hence batch the files into groups of 5.

    Args:
        files_remaining (list): List of full s3 url strings of unprocessed files.

    Returns:
        batches (list): List of lists containing batches of 5 files, per embedded list.
    """

    n = 5
    batches = [
        files_remaining[i : i + n] for i in range(0, len(files_remaining), n)
    ]

    return batches

def fetch_batches(files_remaining):

    batches = get_batch_data(files_remaining)

    return batches

# Sql functions

def generate_sql_query(id_list: list, folder_session: str, paths: list) -> str:
    """Generates the sql query for inserting data into reonomy_silver_processed_files table.

    This function takes the list of ids of the recently processed files, as well as the folder_session and the paths of the files
    that each of those file tables were processed into and generates a SQL query.

    Args:
        id_list (list): List of ids of recently processed files.
        folder_session (str): String holding the time the SilverProcesser was instantiated.
        paths (list): List of strings containing the destination paths of each table within a file batch.

    Returns:
        sql_query (str): The sql query to be inserted into reonomy_silver_processed_files postgres table.
    """

    descriptions_path = paths[0]
    sales_path = paths[1]
    tenants_path = paths[2]
    shapes_path = paths[3]
    contacts_path = paths[4]
    taxes_path = paths[5]

    tuples_to_insert = []

    for file_id in id_list:
        file_tuple = (
            file_id,
            folder_session,
            descriptions_path,
            sales_path,
            tenants_path,
            shapes_path,
            contacts_path,
            taxes_path,
        )
        tuples_to_insert.append(file_tuple)

    tuple_string = str(tuples_to_insert)

    sql_query = f"INSERT INTO reonomy_silver_processed_files(file_id, processed_date, descriptions_path, sales_path, tenants_path, shapes_path, contacts_path, taxes_path) VALUES {tuple_string[1:-1]}"

    return sql_query

def batch_to_sql(batch: list, results: list, folder_session: str) -> str:
    """Processes the results and batch ids to be inserted into the generate_sql_query function.

    Args:
        batch (list): List of s3 url strings for a given batch
        results (list): list of strings containing the results returned by wrangler to_parquet method. Which in turn contains the paths of each table destination.
        folder_session (str): String holding the time the SilverProcesser was instantiated.

    Returns:
        sql_query (str): The sql query to be inserted into reonomy_silver_processed_files postgres table.
    """

    paths = [result["paths"][0] for result in results]

    file_id_list = [file.split("/")[-1] for file in batch]

    cleaned_file_id_list = [file.split(".")[0] for file in file_id_list]

    sql_query = generate_sql_query(cleaned_file_id_list, folder_session, paths)

    return sql_query

def query_sql(query: str, conn: object):
    """Query postgres table.

    Args:
        query (str): Query to be inserted
        conn (object): connection object to postgres instance
    """

    cursor = conn.cursor()
    cursor.execute(query)

    conn.commit()
    conn.rollback()

    logger.debug("Query successful")  # TODO: Add logging


In [11]:
def generate_tables(transformed_batch: object) -> zip:
    """Extracts table attribtes from the SilverTransformer instance into a zip object of tables and table names.

    Args:
        transformed_batch (object): Current batch instance of SilverTransformer

    Returns:
        tables (zip): Zip object of tables, and their associated table names.
    """
    desc_table = transformed_batch.description_table
    sales_table = transformed_batch.sales_table
    tenants_table = transformed_batch.tenants_table
    shapes_table = transformed_batch.shapes_table
    contacts_table = transformed_batch.contacts_table
    taxes_table = transformed_batch.taxes_table

    result_tables = [
        desc_table,
        sales_table,
        tenants_table,
        shapes_table,
        contacts_table,
        taxes_table,
    ]

    table_names = [
        "descriptions",
        "sales",
        "tenants",
        "shapes",
        "contacts",
        "taxes",
    ]

    tables = zip(result_tables, table_names)

    return tables

def transform_batch_data(batch_data: object) -> object:
    """Instantiates SilverTransformer object and transforms batch pandas dataframe object.

    siv_transformer object will transform and validate data, and generate the tables as attributes.

    Args:
        batch_data (pd.Dataframe): dataframe object of unnormalized batch data

    Returns:
        siv_transformer (object): Instance of SilverTransformer which contains different processed tables as attributes.
    """
    siv_transformer = SilverTransformer()

    df = siv_transformer.transform_data(batch_data)

    siv_transformer.generate_tables(df)

    return siv_transformer


In [12]:
def process_batches(batches: list):
    """Iterates through batches and processes each batch using all necessary functions.

    Args:
        batches (list): List of batches containing s3 url id strings of unprocessed files.
    """

    for batch in batches:
        batch_data = extract_batch_data(batch)
        transformed_batch = transform_batch_data(batch_data)
        tables = generate_tables(transformed_batch)
        results = tables_to_parquet(tables)
        sql_query = batch_to_sql(batch, results, self.folder_session)
        query_sql(sql_query, self.conn)

    logger.info("Batches processed")
    conn.close()
    logger.info("Connection closed")

In [25]:
from io import BytesIO
def generate_df_from_batch(files: list) -> object:
    """Extracts JSON file from Google cloud storage and loads into pandas dataframe.
    
    """
    storage_client = storage.Client.from_service_account_json(json_credentials_path='./develop_gcp_service_account_key.json')
    bucket = storage_client.bucket(bucket_name)
    full_df = pd.DataFrame()
    for file in files:
        blob = bucket.blob(file)
        df = pd.read_json(BytesIO(blob.download_as_string()))
        full_df = full_df.append(df)
    
    return full_df

def extract_batch_data(batch: list) -> object:
        """Extract unnormalized data into pandas dataframe for a given batch.

        Args:
            batch (list): List of S3 urls for a given batch to be processed.

        Returns:
            data (pd.DataFrame): Pandas DataFrame containing unnormalized data for a given batch extracted from s3.
        """

        # Get data from batch
        data = generate_df_from_batch(batch)
        data = data.reset_index(drop=True)

        return data

    # Loop through batches

In [59]:
def tables_to_parquet(tables: list, bucket_name: str) -> list:

    results = []
    
    storage_client = storage.Client.from_service_account_json(json_credentials_path='./develop_gcp_service_account_key.json')
    bucket = storage_client.bucket(bucket_name)
    
    table_id = str(uuid.uuid4())

    for table, table_name in tables:
        
        table_name_id = f"test_parquet_1/{table_name}/{table_id}.snappy.parquet"
        
        blob = bucket.blob(table_name_id)
        blob.upload_from_string(table.to_parquet(compression='snappy'))

        results.append(table_name_id)
        
        print(table_name_id)

    return results

# Test

In [16]:
bucket_name = "nexus-reonomy-development-raw"

In [17]:
processed_files = fetch_processed_files() 

In [18]:
all_files = fetch_all_files()

In [19]:
remaining_files = get_remaining_files(all_files, processed_files)

In [20]:
batches = get_batch_data(remaining_files)

In [21]:
batch = batches[0]

In [24]:
batch_data = extract_batch_data(batch)

/var/folders/8p/yf_sl9nj34xbrvxr076rlnl40000gn/T/ipykernel_1793/103747343.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/var/folders/8p/yf_sl9nj34xbrvxr076rlnl40000gn/T/ipykernel_1793/103747343.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/var/folders/8p/yf_sl9nj34xbrvxr076rlnl40000gn/T/ipykernel_1793/103747343.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/var/folders/8p/yf_sl9nj34xbrvxr076rlnl40000gn/T/ipykernel_1793/103747343.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/var/folders/8p/yf_s

In [26]:
transformed_batch = transform_batch_data(batch_data)

In [55]:
tables = generate_tables(transformed_batch)

In [58]:
res = tables_to_parquet(tables, bucket_name)

test_parquet_1/descriptions/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet
test_parquet_1/sales/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet
test_parquet_1/tenants/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet
test_parquet_1/shapes/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet
test_parquet_1/contacts/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet
test_parquet_1/taxes/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet


In [60]:
res

['test_parquet_1/descriptions/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet',
 'test_parquet_1/sales/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet',
 'test_parquet_1/tenants/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet',
 'test_parquet_1/shapes/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet',
 'test_parquet_1/contacts/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet',
 'test_parquet_1/taxes/a6953e2c-5321-49c2-a48b-851c405085fc.snappy.parquet']

# transformers

In [14]:
# from reonomy_validator import ReonomyValidator
import pandas as pd
import logging

logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)


class SilverTransformer:

    """This class holds all the methods and variables to complete the transformations on the reonomy bronze data.

    The purpose is to input the raw JSON reonomy data, and return 6 cleaned tables:
    shapes, property_contacts, tenants, sales, taxes and descriptions. These tables will all contain the MSA for partition
    and the reonomy id of the sites wihtin it for better querying.

    Attributes:
        columns_to_transform (list): This is a list of the column names to transform into pandas dataframes.
        validator: Instance of ReonomyValidator object.
    """

    def __init__(self):
        """Initializes SilverTransformer class."""
        self.columns_to_transform = [
            "shapes",
            "property_contacts",
            "tenants",
            "sales",
            "taxes",
            "description",
        ]
        # self.validator = ReonomyValidator()

    # Active functions

    def transform_data(self, data: object) -> object:
        """Inputs reonomy_id and msa name into unnormalized data frame, and drops unwanted columns.

        This function expects a dataframe containing all columns with embdedded json objects and will do three things:
            - Drop unwanted columns
            - Add reonomy_id and msa to each embedded json object
            - Combined summary and details into new description column

        Args:
            data (dataframe): Unnormalized data frame, containing all embdedded json columns as dataframe columns

        Returns:
            df (dataframe): Unnormalized data frame with dropped columns, new description column and reonomy_id + msa in each column json object
        """

        df = data.copy()
        df["description"] = df[["summary", "details"]].apply(
            self.create_description, axis=1
        )
        df["reonomy_id"] = df["scraping_details"].apply(lambda x: x["property_id"])
        df["msa"] = df["details"].apply(lambda x: x["msa_name"])

        columns_to_keep = self.columns_to_transform + ["reonomy_id", "msa"]
        df = df[columns_to_keep]

        for column in self.columns_to_transform:
            df[column] = df[["reonomy_id", column]].apply(
                lambda x: self.insert_value(x, "reonomy_id"), axis=1
            )
            df[column] = df[["msa", column]].apply(
                lambda x: self.insert_value(x, "msa"), axis=1
            )

        return df

    def generate_tables(self, df: object):
        """Creates six class attributes containing each table type as described in columns_to_transform list.

        Args:
            df (dataframe): Expects a un-normalized dataframe with description as a column and reonomy_id + msa within each embedded json object.
        """

        df_dict = df.to_dict()

        self.description_table = self.handle_descriptions(df_dict["description"])
        # self.description_table = self.validator.descriptions_schema.validate(
        #     self.description_table
        # )
        logger.info("Description generated and validated")

        self.sales_table = self.handle_sales(df_dict["sales"])
        # self.sales_table = self.validator.sales_schema.validate(self.sales_table)
        # logger.info("Sales generated and validated")

        self.tenants_table = self.handle_tenants(df_dict["tenants"])
        # self.tenants_table = self.validator.tenants_schema.validate(self.tenants_table)
        # logger.info("Tenants generated and validated")

        self.shapes_table = self.handle_shapes(df_dict["shapes"])
        # self.shapes_table = self.validator.shapes_schema.validate(self.shapes_table)
        # logger.info("Shapes generated and validated")

        self.contacts_table = self.handle_contacts(df_dict["property_contacts"])
        # self.contacts_table = self.validator.contacts_schema.validate(
        #     self.contacts_table
        # )
        # logger.info("Contacts generated and validated")

        self.taxes_table = self.handle_taxes(df_dict["taxes"])
        # self.taxes_table = self.validator.taxes_schema.validate(self.taxes_table)
        # logger.info("Taxes generated and validated")

    # Handler functions

    def create_description(self, x: object) -> object:
        """Creates description from summary and details columns.

        Args:
            x (two dataframe series): dataframe row contain summary and details data. Passed using axis=1 in df lambda function

        Returns:
            description (dataframe series): combined description series of summary and details columns
        """
        description = {**x[0], **x[1]}

        return description

    def insert_value(self, x: object, value_name: str):
        """Inserts value into embedded json

        Args:
            x (pandas series): two column pandas series containing the value to insert and the embedded json object
            value_name (str): _description_

        Returns:
            current_dict: embedded json with new value inserted
        """
        value = x[0]
        current_dict = x[1]

        current_dict[value_name] = value

        return current_dict

    def handle_descriptions(self, descriptions_dict: dict):
        """Transforms the descriptions dictionary into a normalized pandas tables

        Args:
            descriptions_dict (dict): Dictationary containing descriptions data

        Returns:
            desc_table (dataframe): normalized pandas dataframe containing descriptions data
        """

        description_tbl = pd.json_normalize(descriptions_dict, max_level=0).T
        description_tbl = description_tbl.rename(columns={0: "description"})

        row_length = description_tbl.shape[0]

        desc_table = pd.DataFrame()
        for i in range(row_length):
            d = description_tbl.iloc[i]["description"]
            row = pd.json_normalize(d, max_level=0)
            desc_table = pd.concat([desc_table, row])

        desc_table = desc_table.reset_index(drop=True)

        return desc_table

    def handle_sales(self, sales_dict):
        """Transforms the sales dictionary into a normalized pandas tables

        Args:
            sales_dict (dict): Dictationary containing sales data

        Returns:
            sales_table (dataframe): normalized pandas dataframe containing sales data
        """

        sales_tbl = pd.json_normalize(sales_dict, max_level=0).T
        sales_tbl = sales_tbl.rename(columns={0: "sales"})

        sales_table = pd.DataFrame()

        row_length = sales_tbl.shape[0]

        for i in range(row_length):
            row = sales_tbl.iloc[i]["sales"]

            property_sales_df = pd.json_normalize(
                row,
                max_level=0,
                record_path=["sales"],
                meta=["reonomy_id", "sale_update_time", "msa"],
            )

            sales_table = pd.concat([sales_table, property_sales_df])

        sales_table = sales_table.reset_index(drop=True)

        return sales_table

    def handle_tenants(self, tenants_dict):
        """Transforms the tenants dictionary into a normalized pandas tables

        Args:
            tenants_dict (dict): Dictationary containing tenants data

        Returns:
            tenants_table (dataframe): normalized pandas dataframe containing tenants data
        """

        tenants_tbl = pd.json_normalize(tenants_dict, max_level=0).T
        tenants_tbl = tenants_tbl.rename(columns={0: "tenants"})

        tenants_table = pd.DataFrame()

        row_length = tenants_tbl.shape[0]

        for i in range(row_length):
            row = tenants_tbl.iloc[i]["tenants"]

            tenants_df = pd.json_normalize(
                row, max_level=0, record_path=["tenants"], meta=["reonomy_id", "msa"]
            )

            tenants_table = pd.concat([tenants_table, tenants_df])

        tenants_table = tenants_table.reset_index(drop=True)

        return tenants_table

    def handle_shapes(self, shapes_dict):
        """Transforms the shapes dictionary into a normalized pandas tables

        Args:
            shapes_dict (dict): Dictationary containing shapes data

        Returns:
            shapes_table (dataframe): normalized pandas dataframe containing shapes data
        """

        shapes_tbl = pd.json_normalize(shapes_dict, max_level=0).T
        shapes_tbl = shapes_tbl.rename(columns={0: "shapes"})

        shapes_table = pd.DataFrame()

        row_length = shapes_tbl.shape[0]

        for i in range(row_length):
            row = shapes_tbl.iloc[i]
            if "items" in row["shapes"].keys():

                shapes_df = pd.json_normalize(
                    row, max_level=0, record_path=["items"], meta=["reonomy_id", "msa"]
                )

                shapes_table = pd.concat([shapes_table, shapes_df])

        shapes_table = shapes_table.reset_index(drop=True)

        return shapes_table

    def handle_contacts(self, contacts_dict):
        """Transforms the contacts dictionary into a normalized pandas tables

        Args:
            contacts_dict (dict): Dictationary containing contacts data

        Returns:
            contacts_table (dataframe): normalized pandas dataframe containing contacts data
        """

        contacts_tbl = pd.json_normalize(contacts_dict, max_level=0).T
        contacts_tbl = contacts_tbl.rename(columns={0: "property_contacts"})

        contacts_table = pd.DataFrame()

        row_length = contacts_tbl.shape[0]

        for i in range(row_length):
            row = contacts_tbl.iloc[i]

            contacts_df = pd.json_normalize(row, max_level=0)

            contacts_table = pd.concat([contacts_table, contacts_df])

        contacts_table = contacts_table.reset_index(drop=True)

        return contacts_table

    def handle_taxes(self, taxes_dict):
        """Transforms the taxes dictionary into a normalized pandas tables

        Args:
            taxes_dict (dict): Dictationary containing taxes data

        Returns:
            taxes_table (dataframe): normalized pandas dataframe containing taxes data
        """

        taxes_tbl = pd.json_normalize(taxes_dict, max_level=0).T
        taxes_tbl = taxes_tbl.rename(columns={0: "taxes"})

        taxes_table = pd.DataFrame()

        row_length = taxes_tbl.shape[0]

        for i in range(row_length):
            row = taxes_tbl.iloc[i]["taxes"]

            propert_taxes_df = pd.json_normalize(
                row,
                max_level=0,
                record_path=["taxes"],
                meta=["reonomy_id", "tax_update_time", "msa"],
            )

            taxes_table = pd.concat([taxes_table, propert_taxes_df])

        taxes_table = taxes_table.reset_index(drop=True)

        return taxes_table

In [15]:
from pandera import Column, DataFrameSchema, Check, Index
import logging


# session_logging = logging.getLogger(__name__)
# session_logging.setLevel(logging.DEBUG)

# session_file_handler = logging.FileHandler("session.log")

# session_logging.addHandler(session_file_handler)


class ReonomyValidator:
    """This class loads the schema object from the local schema library  and implements it to be used by the silver transformer.

    Attributes:
        descriptions_schema: pandera with descriptions schema
        sales_schema: pandera object with sales schema
        taxes_schema: pandera object with taxes schema
        contacts_schema: pandera object with contacts schema
        shapes_schema: pandera object with shapes schema
        tenants_schema: pandera object with tenants schema

    """

    def __init__(self):
        """Initializes Reonomy Validator"""
        self.descriptions_schema = self.load_schema("descriptions")
        self.sales_schema = self.load_schema("sales")
        self.taxes_schema = self.load_schema("taxes")
        self.contacts_schema = self.load_schema("contacts")
        self.shapes_schema = self.load_schema("shapes")
        self.tenants_schema = self.load_schema("tenants")

    def load_schema(self, table_type: str) -> object:
        """Fetches associated schema object based on table name

        Args:
            table_type (str): Name of the schema type object to fetch

        Returns:
            schema (obj): Schema object of table_type
        """
        file_name = f"schemas/{table_type}.yaml"
        schema = DataFrameSchema.from_yaml(file_name)
        return schema

/Users/alexandergirardet/miniforge3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
